In [1]:
from langchain import OpenAI, LLMChain
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
import os
import uuid
import openai
from langchain.chat_models import ChatOpenAI
COLLECTION_NAME = "DocumentsDB_Langchain"
openai.api_key = os.environ["OPENAI_API_KEY"]
llm=OpenAI()

## Qdrantに接続

In [2]:
from qdrant_client import QdrantClient,models
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant

client = QdrantClient(host='qdrant', port=6333)
embeddings = OpenAIEmbeddings()
print("Connection successful:", client)

db = Qdrant(
    client=client, collection_name=COLLECTION_NAME, 
    embeddings=embeddings
)


Connection successful: <qdrant_client.qdrant_client.QdrantClient object at 0x7fe3ddf10a90>


## 関連情報探索

In [52]:
query = "ビジネス光ネットサービスの契約概要について教えて"
docs = db.similarity_search(query=query,k=10)
for d in docs:
    print(d.page_content)
# related_info = retriever.get_relevant_documents(query=query)

(注)業務の遂行上必要な範囲での利用には、光ネットサービス契約者に係る情報を当社の業務を委託している者に提供する場合を含みます。
4当社が別に定める光ネットサービス当社は、下記の契約約款に定める光ネットサービスを当社が別に定める光ネットサービスとして扱います。
2前項の確認に際して、光ネットサービス契約者から請求があったときは、当社は、光ネットサービス取扱局において試験を行い、その結果を光ネットサービス契約者に通知します。
3光ネットサービス契約者は、前2項に規定する接続について、第1項の書面に記載した事項を変更しようとするときは、当社所定の書面によりその変更の請求をしていただきます。この場合、当社は、前項の規定に準じて取り扱います。
第1章総則(約款の適用)第1条当社は、このビジネス光ネットサービス契約約款(料金表を含みます。以下「約款」といいます。)を定め、これにより光ネットサービス(当社がこの約款以外の契約約款及び料金表を定め、それにより提供するものを除きます。)を提供します。
2当社は、当社が指定する光ネットサービス取扱所において、光ネットサービスを利用する上で参考となる、別記13に定める事項を記載した技術資料を閲覧に供します。
(品目及び種別等の変更)第14条光ネットサービス契約者は、当社に対し、当社が別に定めるところにより光ネットサービスの品目及び種別等の変更を請求することができます。
4光ネットサービス契約者は、第1項及び第2項に規定する接続を廃止しようとするときは、あらかじめ書面により光ネットサービス取扱所に通知していただきます。
区分内容(1)事務手数料等に係る料金の適用ア光ネットサービス契約の申込みをし、その承諾を受けたときに契約事務手数料を適用します。
2前項の場合において、当社は譲り受けた債権を当社が提供する光ネットサービスの料金とみなして取り扱います。


In [ ]:
def relate_search(k=3,filter=None):
    docs = db.similarity_search(query=query,k=10,filter=None)
    return [d.page_content for d in docs]

In [20]:
a = [(1,2),(3,4)]
for i,j in a:
    print(i,j)

1 2
3 4


## memoryの実装
- ConversationBufferWindowMemory  
 直前数回のやり取りをプロンプトに入れる
- VectorStore-Backed Memory  
  プロンプトをベクトル化し、近いやり取りを埋め込む

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferWindowMemory,VectorStoreRetrieverMemory
from langchain.schema import messages_from_dict, messages_to_dict
from langchain import LLMChain

In [ ]:
memory=ConversationBufferWindowMemory(k=3)

def save_memory(memory,query,response):
    memory.chat_memory.add_user_message(query)
    memory.chat_memory.add_ai_message(response)

## promptTemplate作成

In [ ]:
template = """
{chat_history}
質問：{query}以下の関連情報をもとに答えてください。
関連情報
1.{relate_info1}
2.{relate_info2}
3.{relate_info3}
"""

In [18]:
def create_template(relate_info):
    input_val = ["chat_history","query"]
    template = "{chat_history}\n質問：{query}\n関連情報\n"
    for i,info in enumerate(relate_info):
        template += f"{i+1}.{info}\n"
    template += "解答："
    return PromptTemplate(input_variables=input_val,template=template)

template = create_template(["b","b"])
prompt = template.format(chat_history="",query="a")
print(prompt)


質問：a
関連情報
1.b
2.b
解答：


## 対話

In [3]:
def chat(query,t):
    related_data = relate_search(query,k=10)
    template = create_template(related_data)
    memory = ConversationBufferWindowMemory(memory_key="chat_history",k=3)
    memory.chat_memory.add_user_message("hi!")
    memory.chat_memory.add_ai_message("what's up?")
    llm_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    memory=memory,
    verbose=True,
    )
    return llm_chain

In [19]:
from langchain.callbacks import get_openai_callback

template = create_template(["9月24日は日曜日です"])
memory = ConversationBufferWindowMemory(memory_key="chat_history",k=3)
memory.chat_memory.add_user_message("今から日にちを言うのでその日の曜日を答えてください")
memory.chat_memory.add_ai_message("分かりました")
llm_chain = LLMChain(
    llm=OpenAI(),
    prompt=template,
    memory=memory,
    verbose=True,
    )
with get_openai_callback() as cb:
    response = llm_chain("9月25日は？")
print(response["text"])
print(type(cb))
print(cb)
print(cb.__dict__)



> Entering new LLMChain chain...
Prompt after formatting:
Human: 今から日にちを言うのでその日の曜日を答えてください
AI: 分かりました
質問：9月25日は？
関連情報
1.9月24日は日曜日です
解答：

> Finished chain.
月曜日
<class 'langchain.callbacks.openai_info.OpenAICallbackHandler'>
Tokens Used: 115
	Prompt Tokens: 108
	Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $0.0023
{'successful_requests': 1, 'total_cost': 0.0023, 'total_tokens': 115, 'prompt_tokens': 108, 'completion_tokens': 7}


In [13]:
LLMChain.__dict__

mappingproxy({'__config__': pydantic.v1.config.Config,
              '__fields__': {'memory': ModelField(name='memory', type=Optional[BaseMemory], required=False, default=None),
               'callbacks': ModelField(name='callbacks', type=Union[List[langchain.callbacks.base.BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None),
               'callback_manager': ModelField(name='callback_manager', type=Optional[BaseCallbackManager], required=False, default=None),
               'verbose': ModelField(name='verbose', type=bool, required=False, default_factory='<function _get_verbosity>'),
               'tags': ModelField(name='tags', type=Optional[List[str]], required=False, default=None),
               'metadata': ModelField(name='metadata', type=Optional[Mapping[str, Any]], required=False, default=None),
               'prompt': ModelField(name='prompt', type=BasePromptTemplate, required=True),
               'llm': ModelField(name='llm', type=BaseLangu